## How to grab a Cloud-Optimized Geotiff and Visualise it Interactively

In this tutorial, we will show how someone can access the open EO data from open STAC catalogs and visualize them interactively using the Python ecosystem. To achieve this, we need the following packages within our Python environment.

- [xcube](https://xcube.readthedocs.io/en/latest/)
- [xarray](https://docs.xarray.dev/en/stable/)
- [pandas](https://pandas.pydata.org/docs/)
- [leafmap](https://leafmap.org/)
- [shapely](https://shapely.readthedocs.io/en/stable/manual.html)
- [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/)
- [pystac_client](https://pystac-client.readthedocs.io/en/stable/)

If you have these packages installed in your Python environment, then let's get started.

### Importing the tools we are going to use 

In [1]:
from leafmap import Map
from pystac_client import Client
from shapely.geometry import shape
from ipyleaflet import DrawControl, LayersControl
import rasterio
from pandas import date_range
from xarray import open_dataset, concat
from xcube.webapi.viewer import Viewer

### Selection of the area of interest for global layers

Since personal computers don't have extravagant memory, we don't want to encounter memory usage issues. We need to pick a region we are interested in to load and visualize from global layers. Let's pick South America to visualize without downloading data on our local drive.

In [2]:
# define the map properties 
map_props = dict(draw_control=False, measure_control=False, fullscreen_control=False)

map = Map(**map_props)
map.add_basemap('HYBRID') # adding a basemap
draw_control = DrawControl()
draw_control.rectangle = { 'shapeOptions': {'color':'#ff0000', 'fillOpacity':0, 'opacity':1}}
map.add_control(draw_control)
map


Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

#### After selecting the area of interest, we can extract the bounds we selected from the map

In [3]:
geometry = shape(draw_control.data[-1]['geometry'])
bounds = geometry.bounds
print(f'Bounds : {bounds}')

Bounds : (-100.390035, -57.073257, -31.857913, 18.75609)


### Let's get global data layers from open-source stac catalogs

You can select a global datalayer collection from either [OpenLandMap STAC](https://stac.openlandmap.org) or [EcoDataCube STAC](https://stac.ecodatacube.eu). These two open-source data catalogs have a dedicated web application to show the data layers presented in the STAC catalog. You can access these web applications via [OpenLandMap](https://openlandmap.org/) and [EcoDataCube](https://ecodatacube.eu/). If you are a [QGIS](https://qgis.org/) user, you can download and install the dedicated [QGIS Plugin](https://github.com/Open-Earth-Monitor/oemc-qgis-plugin) to explore the data layers in these STAC catalogs. 

In [4]:
# define the catalog route and the id's of the collection and item we are interested in
# I selected the yearly water vapor data layers
catalog_url = "https://s3.eu-central-1.wasabisys.com/stac/openlandmap/catalog.json"
target_collection = "wv_mcd19a2v061.seasconv.m.yearly"
target_item = "wv_mcd19a2v061.seasconv.m.yearly_p75_1km_s"

file_store = [] # storage for the URL of the rasters

catalog = Client.open(catalog_url)

for collection in catalog.get_collections():
    if collection.id == target_collection:
        items = collection.get_all_items()
        for item in items:
            for key in item.to_dict()['assets'].keys():
                if key == target_item:
                    file_store.append(item.to_dict()['assets'][key]['href'])
        

In [5]:
file_store

['https://s3.openlandmap.org/arco/wv_mcd19a2v061.seasconv.m.yearly_p75_1km_s_20000101_20001231_go_epsg.4326_v20230619.tif',
 'https://s3.openlandmap.org/arco/wv_mcd19a2v061.seasconv.m.yearly_p75_1km_s_20010101_20011231_go_epsg.4326_v20230619.tif',
 'https://s3.openlandmap.org/arco/wv_mcd19a2v061.seasconv.m.yearly_p75_1km_s_20020101_20021231_go_epsg.4326_v20230619.tif',
 'https://s3.openlandmap.org/arco/wv_mcd19a2v061.seasconv.m.yearly_p75_1km_s_20030101_20031231_go_epsg.4326_v20230619.tif',
 'https://s3.openlandmap.org/arco/wv_mcd19a2v061.seasconv.m.yearly_p75_1km_s_20040101_20041231_go_epsg.4326_v20230619.tif',
 'https://s3.openlandmap.org/arco/wv_mcd19a2v061.seasconv.m.yearly_p75_1km_s_20050101_20051231_go_epsg.4326_v20230619.tif',
 'https://s3.openlandmap.org/arco/wv_mcd19a2v061.seasconv.m.yearly_p75_1km_s_20060101_20061231_go_epsg.4326_v20230619.tif',
 'https://s3.openlandmap.org/arco/wv_mcd19a2v061.seasconv.m.yearly_p75_1km_s_20070101_20071231_go_epsg.4326_v20230619.tif',
 'https:

We can see the time coverage of the data layers from their names and the time coverage of each raster file is __1 year__.  If you want to make a deep-dive into the naming convention of this stac [the perfect spot](https://openlandmap.github.io/book/#the-file-naming-convention) is here for it.

In [6]:
# Converting the bounding box of the South America to raster pixels for the desired data layers

with rasterio.open(file_store[0]) as src:
    window = rasterio.windows.from_bounds(bounds[0],bounds[1],bounds[2],bounds[3],src.transform)
    stats = src.statistics(1)

window

Window(col_off=9553.196182127847, row_off=8233.669529346782, width=8223.8549689542, height=9099.522003980881)

In [7]:
stats

Statistics(min=29.0, max=6931.0, mean=1747.3372062931478, std=1155.7961700334056)

In [8]:
# Since we are going to create a datacube while reading and concatenating the rasters, we need a function to do it in a tidy manner

def read_raster(file_url, window=None):
    if window:
        d = open_dataset(file_url, engine='rasterio').isel(
            x=slice(int(window.col_off), int(window.col_off + window.width)),
            y=slice(int(window.row_off), int(window.row_off + window.height))
        )
    else:
        d = open_dataset(file_url, engine='rasterio')
    d = d.expand_dims(time = date_range(file_url.split('_')[5], file_url.split('_')[6], freq='Y'))
    return d

In [9]:
first_run = True
for f in file_store:
    if first_run:
        datacube = read_raster(f, window)
        first_run = False
    else:
        d = read_raster(f,window)
        datacube = concat([datacube, d], dim='time')

In [10]:
## setting up the XCUBE viewer to view and interact with the data 
viewer = Viewer(
    server_config = {
        'Styles': [
            {
                'Identifier' : 'mydata',
                'ColorMappings': {
                    'band_data': {
                        'ValueRange':[stats.min, stats.max],
                        'ColorBar':'plasma_r'
                    }
                }
            }
        ]
    }
)

In [11]:
viewer.add_dataset(datacube, style='mydata')
viewer.show()

404 GET /viewer/config/config.json (127.0.0.1): xcube viewer has not been been configured
404 GET /viewer/config/config.json (127.0.0.1) 2.85ms
